# Test out `topp.py`, `util.py`, `io.py`

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import src.io as io
import src.topp as topp
import src.util as util
plt.rc('axes', unicode_minus=False)  # Glyph 8722

In [ ]:
strokes = io.load_log('logs/G.txt')
for stroke in strokes:
    print(stroke.shape)

In [ ]:
# Plot strokes
strokes = util.clean_strokes(strokes)
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for t, xy in util.strokes2txs(strokes):
    xydot, xyddot = util.derivatives(t, xy)
    axes[0].plot(*xy.T)
    axes[1].plot(t, np.linalg.norm(xydot, axis=1))
    axes[2].plot(t, np.linalg.norm(xyddot, axis=1))
axes[0].axis('equal')
axes[0].set_title('xy')
axes[1].set_title('Speed vs time')
axes[2].set_title('||Acceleration|| vs time');

In [ ]:
# Do TOPP-RA
stroke = topp.Stroke(strokes[0], clean=True)
stroke.spline_interp()
stroke.retime(vmax=1, amax=5)

ts, xs, xds, xdds = stroke.sample_retimed(N=200)

In [ ]:
# Plot TOPP-RA results
fig, axes = plt.subplots(3, 2, figsize=(5, 6))
stroke.plot_xva(axes)
fig.tight_layout()

In [253]:
# Create html animation
HTML(stroke.create_html_anim('test1.html'))

# Investigate the data format of `toppra.SplineInterpolator`
(to reverse-engineer)

In [ ]:
coeff = stroke.path.cspl.c
coeffv = stroke.path.cspld.c
breakpts = stroke.path.cspl.x
print(coeff.shape) # (degree, numsegments, xy)
print(breakpts.shape) # (numsegments+1)

In [ ]:
print(coeff.shape, coeffv.shape) # coeffv should have degree 1 less than coeff
print(coeff[:, 10, 0])
print(coeffv[:, 10, 0])
print(coeff[:, 10, 0] * np.arange(3, -1, -1)) # manually take the derivative to compare to coeffv

In [ ]:
# Now try to manually evaluate the piecewise polynomial!
t = np.array([6e-4, 0.2, 0.4, 0.6, 1])
expected = stroke.path(t)
print(expected)

# manually evaluate the spline
t2 = t.reshape(-1, 1)
cond = np.logical_and(t2 >= breakpts[:-1].reshape(1, -1), t2 < breakpts[1:].reshape(1, -1))
xy = []
for t_, col in zip(t, cond):
    coeff_ = coeff[:, col, :].squeeze()
    N = coeff_.shape[0] - 1
    t_ = t_ - breakpts[:-1][col]
    tpow = np.power(t_, np.arange(N, -1, -1)).reshape(1, -1)
    xy.append(tpow @ coeff_)
actual = np.array(xy).squeeze()
print(actual)
print(expected - actual)

In [ ]:
# Implemented the above code in `topp.manually_evaluate_spline`
# Test `topp.manually_evaluate_spline`
t = np.array([6e-4, 0.2, 0.4, 0.6, 1])
expected = stroke.path(t)
actual = topp.manually_evaluate_spline(t, stroke.path)
np.testing.assert_allclose(actual, expected, rtol=1e-15, atol=1e-15)

# Create data for c++ unit tests

In [254]:
xc = [1, 2, 3, 4]
yc = [5, 6, 7, 8]
t = 0.1
xdc, ydc = np.polyder(xc), np.polyder(yc)
xddc, yddc = np.polyder(xdc), np.polyder(ydc)
for t in [0.05, 0.09999999]:
    print(f'EXPECT_XVA_EQUAL({t}, 1e-6,             // t, tol')
    print(f'                 {np.polyval(xc, t):.6f}, {np.polyval(yc, t):.6f},    // x')
    print(f'                 {np.polyval(xdc, t):.6f}, {np.polyval(ydc, t):.6f},    // v')
    print(f'                 {np.polyval(xddc, t):.6f}, {np.polyval(yddc, t):.6f});  // a')

EXPECT_XVA_EQUAL(0.05, 1e-6,             // t, tol
                 4.155125, 8.365625,    // x
                 3.207500, 7.637500,    // v
                 4.300000, 13.500000);  // a
EXPECT_XVA_EQUAL(0.09999999, 1e-6,             // t, tol
                 4.321000, 8.765000,    // x
                 3.430000, 8.350000,    // v
                 4.600000, 15.000000);  // a


In [255]:
for i in range(4):
    b = breakpts[i+1]
    c = coeff[:, i, :].squeeze()
    row2str = lambda row: ', '.join(f'{n:.6f}' for n in row)
    print(f'spline.add_segment({b}, {{{{ {{{{ {row2str(c[:, 0])} }}}}, {{{{ {row2str(c[:, 1])} }}}} }}}});')

spline.add_segment(0.009803361344537943, {{ {{ -518.289381, 5.080978, 0.000000, 0.521429 }}, {{ 1261.529542, -1.749690, 0.000000, -0.187755 }} }});
spline.add_segment(0.021008435681965202, {{ {{ 578.314546, -10.161956, -0.049811, 0.521429 }}, {{ -2152.101674, 35.352000, 0.329416, -0.186735 }} }});
spline.add_segment(0.03386929541047199, {{ {{ -840.105840, 9.278216, -0.059713, 0.520408 }}, {{ 1475.419016, -36.991378, 0.311046, -0.181633 }} }});
spline.add_segment(0.04813707175177775, {{ {{ 682.343536, -23.135234, -0.237926, 0.519388 }}, {{ -793.512742, 19.934093, 0.091675, -0.180612 }} }});


In [256]:
for t in [0.005, 0.01, 0.015, breakpts[2], 0.035, 0.045]:
    print(f'EXPECT_XVA_EQUAL({t}, 1e-6,            // t, tol')
    print(f'                 {row2str(stroke.path.eval(t))},  // x')
    print(f'                 {row2str(stroke.path.evald(t))},  // v')
    print(f'                 {row2str(stroke.path.evaldd(t))});  // a')
t = breakpts[4]
print(f'EXPECT_XVA_EQUAL({0.05}, 1e-6,            // t, tol')
print(f'                 {row2str(stroke.path.eval(t))},  // x')
print(f'                 {row2str(stroke.path.evald(t))},  // v')
print(f'                 {row2str(stroke.path.evaldd(t))});  // a')

EXPECT_XVA_EQUAL(0.005, 1e-6,            // t, tol
                 0.521491, -0.187641,  // x
                 0.011938, 0.077118,  // v
                 -5.386725, 34.346506);  // a
EXPECT_XVA_EQUAL(0.01, 1e-6,            // t, tol
                 0.521418, -0.186669,  // x
                 -0.053740, 0.343069,  // v
                 -19.641598, 68.164881);  // a
EXPECT_XVA_EQUAL(0.015, 1e-6,            // t, tol
                 0.520976, -0.184370,  // x
                 -0.108574, 0.522486,  // v
                 -2.292162, 3.601831);  // a
EXPECT_XVA_EQUAL(0.021008435681965202, 1e-6,            // t, tol
                 0.520408, -0.181633,  // x
                 -0.059713, 0.311046,  // v
                 18.556433, -73.982756);  // a
EXPECT_XVA_EQUAL(0.035, 1e-6,            // t, tol
                 0.519090, -0.180484,  // x
                 -0.287627, 0.133711,  // v
                 -41.641294, 34.484815);  // a
EXPECT_XVA_EQUAL(0.045, 1e-6,            // t, tol
         